In [1]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import dotenv_values
import sqlite3
import sqlite_utils
import time
import statistics


In [2]:
mid_kr_puuid = "yke-_ZuAP9wM6s3tpe-FnVWLlnwI4qLuLRzz933HfpI_jzKcDa1YwXbWGpFC7WJek_rv4x_kXk2Y-A"

In [3]:
conn = sqlite3.connect('koren_soloq_db.db')
cursor = conn.cursor()
match_end_data = cursor.execute('''SELECT metadata, info FROM MATCH_DATA''').fetchall()
match_timeline_data = cursor.execute('''SELECT metadata, timelineinfo FROM MATCH_DATA''').fetchall()

conn.close()

In [4]:
#Filter only mid role games
list_match_end_data_whereMid = []
mid_matchs = []
for idx_match, match in enumerate(match_end_data) :
    match_info = json.loads(match[1])
    for idx, participant in enumerate(match_info['participants']):
        if (participant['puuid'] == mid_kr_puuid) and (participant['teamPosition'] == "MIDDLE"):
            list_match_end_data_whereMid.append(match)
            mid_matchs.append(idx_match)

list_match_timeline_data_whereMid = [match_timeline_data[i] for i in mid_matchs]

In [5]:
print(len(match_end_data))
print(len(list_match_end_data_whereMid))
print(len(list_match_timeline_data_whereMid))

600
505
505


In [77]:
#Info after game
kda_list = []
cs_min = []
for match in list_match_end_data_whereMid :


    match_metadata = json.loads(match[0])
    for idx, player_puuid in enumerate(match_metadata['participants']):
        if player_puuid == mid_kr_puuid:
            index_qing = idx

    match_info = json.loads(match[1])

    #KDA
    kills = match_info['participants'][index_qing]['kills']
    assists = match_info['participants'][index_qing]['assists']
    deaths = match_info['participants'][index_qing]['deaths']
    if deaths == 0 : deaths = 1
    kda_list.append((kills+assists)/deaths)

    #CS/MIN
    monster_cs = match_info['participants'][index_qing]['neutralMinionsKilled']
    minion_cs = match_info['participants'][index_qing]['totalMinionsKilled']
    cs_min.append((monster_cs + minion_cs) /  (match_info['gameDuration'] //60))




In [78]:
json.loads(list_match_end_data_whereMid[0][1])['participants'][0]['puuid']

'kKSPluUio3QQcAttyUdjul-d2m3rPrCRxxf4llXyzcwkwdtpnZ6PGHFY_NH7IQfgbgf-XPko4bZE8A'

In [79]:
list_match_end_data_whereMid[0][0]

'{"dataVersion": "2", "matchId": "KR_7329575443", "participants": ["kKSPluUio3QQcAttyUdjul-d2m3rPrCRxxf4llXyzcwkwdtpnZ6PGHFY_NH7IQfgbgf-XPko4bZE8A", "b2UKF96Wyn0vtAH6O-p7OPO8Txkqr1Z0ubt6B925KqRRtD0d__zXp9-o9zglb0NEPAwtkCM39e1LGg", "yke-_ZuAP9wM6s3tpe-FnVWLlnwI4qLuLRzz933HfpI_jzKcDa1YwXbWGpFC7WJek_rv4x_kXk2Y-A", "c0mt04o4UAxTdjVuoo4GsH3XHBKT6Coyo16wOy0GZnq75iUgJvynzorBfi5AhNoQDEATkYsFuIRVsA", "QdCXe74ryc7a4jm50Iwsvla4_s8Bz2Wctri4YUNkeGb02M4_BHl_4P7AcSgfsS3rHRAPDuytNJ1LMw", "uNZ2ByMMKnq-fXxnn5kM-InjI6heHm8LNGstIx27urEEv1itKlfesi0yHdxjlsDJQ3F1b3Juju-yRQ", "F6DhO643BOZAy3DKIWgOypHFCrHJIushJxmZgGezcQC-hqjIgx3jBo3-ONPbSNY0yqwYr6oNkPIIsA", "SIm-goi6I4mmdcs5H14KCVVZ84gAK-cftANxXy06WZiNITwgSbZ49pRo4GDXD_YhkH6RFCaeTI3ZcQ", "X3hCz-3cZu_MalEmaswX1L_02TZWgv-sQ8rhCsikD-E6cpi-Z5WNkEcoD8MuzKD1O-4iRtcyFtH6kg", "P0w1miqQsyHEpCP3OQAnPnCezz5U1UnHFNW2jcdDm7brvU4eQrAXnyJDvbTHlvXNuxRM6p-friLx0w"]}'

In [80]:
#Info timeline
gold_diff_15_list = []
for matchIndex, match in enumerate(list_match_timeline_data_whereMid):


    match_metadata = json.loads(match[0])
    for idx, player_puuid in enumerate(match_metadata['participants']):
        if player_puuid == mid_kr_puuid:
            index_qing = idx

    
    for idx, participant in enumerate(json.loads(list_match_end_data_whereMid[matchIndex][1])['participants']):
        if (participant['teamPosition'] == "MIDDLE") and (participant['puuid']) != mid_kr_puuid:
            index_ennemy = idx
           

    match_info = json.loads(match[1])
    
    if len(match_info['frames']) > 16 :
        qing_gold_15 = match_info['frames'][15]['participantFrames'][str(index_qing+1)]['totalGold']
        ennemy_gold_15 = match_info['frames'][15]['participantFrames'][str(index_ennemy+1)]['totalGold']
        gold_diff_15_list.append(qing_gold_15 - ennemy_gold_15)

# gold diff
# forward %

In [81]:
#attention il a des games autofills
print(statistics.mean(kda_list))
print(statistics.mean(cs_min))
print(statistics.mean(gold_diff_15_list))

4.438730873087309
7.886367344876574
236.53493013972056
